In [ ]:
print("Hey")

In [ ]:
%pwd

In [ ]:
cd ..

In [ ]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#Extracting the data from pdf

def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    

    documents=loader.load()
    return documents
    

In [ ]:
extracted_data=load_pdf_file(data='Data/')

In [ ]:
#extracted_data

In [ ]:
#split the data into text chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print('Length of the text chunks is : ',len(text_chunks))

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [ ]:
#Download the embeddings from HugginFace

def download_hugging_face_embedings():
    embeddings=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings=download_hugging_face_embedings()

In [ ]:
query_result=embeddings.embed_query("Hello world")
print('Lenth ',len(query_result))

In [ ]:
# query_result

In [19]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc=Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        
        )
)
